In [1]:
import pandas as pd
file = "data/4import_Database_Austria_March2018.xlsx"
df = pd.read_excel(file, "RESEARCH")[2:]

In [2]:
df.fillna(value=False, inplace=True)

In [3]:
df.head()

,*SiteID,*Research Activity ID,Start Date,End Date,Responsible Researcher,Responsible Institution,When was the data-set generated?,Activity-ID,Research Question,Comment,...,SUB SUB-TABLE SUPPORTING ANALYSIS - CHEMICAL ANALYSIS,SUB SUB-TABLE SUPPORTING ANALYSIS - CERAMIC ANALYSIS,SUB SUB-TABLE SUPPORTING ANALYSIS - PALYNOLOGY,SUB SUB-TABLE SUPPORTING ANALYSIS - ORGANIC RESIDUE ANALYSIS,SUB SUB-TABLE SUPPORTING ANALYSIS - VITREOUS MATERIAL ANALYSIS,SUB SUB-TABLE SUPPORTING ANALYSIS - OTHER,SUB-TABLE OTHER,EntityID,Public,EnteredBy
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,OE_08001,OE_08001.R0001,2016?,2017 still in progress (I-A-D),GIS Stmk,GIS Stmk - A17 Regional and Regional Developme...,False,False,False,False,...,False,False,False,False,False,False,False,OE_08001.E0001,yes,Raggam Patricia
4,OE_08001,OE_08001.R0002,2016,2017 still in progress (I-A-D),Mele Marko,"UMJ - Universalmuseum Joanneum, Archäologiemus...",False,False,False,False,...,False,False,False,False,False,False,False,OE_08001.E0001,yes,Raggam Patricia
5,OE_08001,OE_08001.R0003,2016,2017 still in progress (I-A-D),False,False,False,False,False,False,...,False,False,False,False,False,False,False,OE_08001.E0001?,yes,Raggam Patricia
6,OE_08001,OE_08001.R0004,2016,2017 still in progress (I-A-D),Mele Marko,"UMJ - Universalmuseum Joanneum, Archäologiemus...",False,False,False,False,...,False,False,False,False,False,False,False,OE_08001.E0001,yes,Raggam Patricia


In [14]:
for i, row in df.iterrows():
    if row['*SiteID']:
        temp_research, _ = ResearchEvent.objects.get_or_create(
            identifier=row['*Research Activity ID']
        )
        temp_research.name = row['*Research Activity ID']
        temp_research.start_date=row['Start Date']
        temp_research.end_date=row['End Date']
        temp_research.save()
        temp_site, _ = Site.objects.get_or_create(
            identifier=row['*SiteID']
        )
        if row['Responsible Researcher']:
            for x in row['Responsible Researcher'].split(','):
                if x:
                    temp_pers, _ = Person.objects.get_or_create(
                        written_name=x
                    )
                    temp_research.responsible_researcher.add(temp_pers)
        if row['Responsible Institution']:
            for x in row['Responsible Institution'].split(','):
                if x:
                    temp_inst, _ = Institution.objects.get_or_create(
                        written_name=x
                    )
                    temp_research.responsible_institution.add(temp_inst)
        if row['Public'] == "yes":
            temp_research.public = True
        temp_research.save()
        if row['Research Method']:
            temp_method,_ = SkosConcept.objects.get_or_create(
                pref_label=row['Research Method']
            )
            temp_research.research_method.add(temp_method)
        temp_site.information_source.add(temp_research)
        temp_site.save()